In [1]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.5
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
import tensorflow as tf
from glob import glob
import matplotlib.pyplot as plt

In [4]:
dataset_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/'
training_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/train/'
validation_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/val/'
test_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/data/test/'
model_path = '/content/drive/My Drive/Colab Notebooks/cotton_disease/inception_model.h5'

IMAGE_SIZE = [224, 224,3]
EPOCHS = 30
TARGET_SIZE = (224,224)
BATCH_SIZE = 16

In [5]:
# useful for getting number of output classes
folders = glob(training_path+'*')
print(len(folders))

4


In [7]:
# importing the resnet and using the imagenet weights
# Here we will be using imagenet weights

resnet_50 = ResNet50(include_top=False,weights='imagenet',input_shape=IMAGE_SIZE)

In [8]:
# don't train existing weights
for layer in resnet_50.layers:
  layer.trainable = False

In [9]:
# our layers - you can add more if you want
x_network = Flatten()(resnet_50.output)
# adding the prediction layer
prediction_layer = Dense(len(folders),activation='softmax')(x_network)

In [11]:
# create a model object
model = Model(inputs = resnet_50.input , outputs = prediction_layer)
model.summary()

Model: "functional_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
_______________________________________________________________________________________

In [12]:
# compiling the model
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [13]:
# creating callbacks to save best model
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath= model_path,
                                                               save_weights_only='False',
                                                               monitor='val_accuracy',
                                                               save_best_only=True)


In [14]:
class MyCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self,epoch,logs = None):
    k = list(logs.keys())
    # print(k)
    if(logs.get('val_accuracy') > 0.99):
      print('Required accuracy reached , so stopping training.....!!')
      self.model.stop_training = True
  

In [15]:
# creating datagenerators

# train datagenerator
train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

# test datagenerator
test_datagen = ImageDataGenerator(rescale= 1./255)